# Main Content
- Train, visualize, and make predictions with Decision Trees.
- CART training algorithm.
- Regularize trees and use them for regression tasks.
- Limitations of Decision Trees.

# Training and Visualizing a Decision Tree
First build one and take a look at how Decision Trees make predictions.

In [1]:
import sklearn
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:,2:] # petal length and width
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

Visualize the trained Decision Tree by using the `export_graphviz()` method to output a grph definition file called `iris_tree.dot`, then convert this dot file to a variety of formats such as PDF or PNG using the `dot` command-line tool from the `graphviz` package.
`dot -Tpng iris_tree.dot -o iris_tree.png`. **First you need to install graphviz package and configure the system path.**

In [5]:
from sklearn.tree import export_graphviz

export_graphviz(
        tree_clf,
        out_file = 'output/chapter5/iris_tree.dot',
        feature_names=iris.feature_names[2:],
        rounded=True,
        filled=True
)

![iris_tree](output/chapter5/iris_tree.png)

# Make Predictions
I think its no need to explain the process of prediction.

**One of the many qualities of Decision Trees is that they require very little data preparation. In particular, they don't require feature scaling or centering at all.**

